In [90]:
import copy
import datetime as dt
import importlib # needed so that we can reload packages
import matplotlib.pyplot as plt
import os, os.path
import numpy as np
import pandas as pd
import pathlib
import sys
import time
from typing import Union
import warnings
from datetime import datetime
warnings.filterwarnings("ignore")

##  IMPORT SISEPUEDE EXAMPLES AND TRANSFORMERS

from sisepuede.manager.sisepuede_examples import SISEPUEDEExamples
from sisepuede.manager.sisepuede_file_structure import SISEPUEDEFileStructure
import sisepuede.core.support_classes as sc
import sisepuede.transformers as trf
import sisepuede.utilities._plotting as spu
import sisepuede.utilities._toolbox as sf

sys.path.append(os.path.join('../', 'utils'))

from TransformationUtils import ExcelYAMLHandler, StrategyCSVHandler
from GeneralUtils import GeneralUtils

In [91]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Initial Set up

Make sure to edit the config yaml under croatia/config_files/croatia_config.yaml

You can also create a new config yaml



In [92]:
g_utils = GeneralUtils()

In [93]:
# Set up dir paths

dir_cur = pathlib.Path(os.getcwd())
dir_project = dir_cur.parent
dir_data = dir_cur.joinpath("data")
dir_config = dir_cur.joinpath("config_files")
dir_transformations = dir_cur.joinpath("transformations")
dir_utils = dir_project.joinpath("utils")
strategies_definitions_file_path = dir_transformations.joinpath("strategy_definitions.csv")
strategy_mapping_file_path = dir_utils.joinpath("strategy_mapping.yaml")

In [94]:
# Load config file, double check your parameters are correct

yaml_file_path = os.path.join(dir_config, "uganda_config.yaml")
config_params = g_utils.get_yaml_values(yaml_file_path)

country_name = config_params['country_name']
ssp_input_file_name = config_params['ssp_input_file_name']
ssp_transformation_cw = config_params['ssp_transformation_cw']


Country Name: uganda
SSP Input File Name: sisepuede_inputs_uganda.csv
SSP Transformation CW: ssp_uganda_transformation_cw.xlsx


In [95]:
# Set up SSP objects

fp_data_base = dir_data.joinpath(ssp_input_file_name)

file_struct = SISEPUEDEFileStructure()

matt = file_struct.model_attributes
regions = sc.Regions(matt)
time_periods = sc.TimePeriods(matt)

### Making sure our input file has the correct format and correct columns
We use an example df with the complete fields and correct format to make sure our file is in the right shape

In [96]:
##  BUILD BASE INPUTS

df_inputs_raw = pd.read_csv(fp_data_base)

# pull example data to fill in gaps
examples = SISEPUEDEExamples()
df_inputs_example = examples.input_data_frame

In [97]:
df_inputs_example.head()

,region,time_period,avgload_trns_freight_tonne_per_vehicle_aviation,avgload_trns_freight_tonne_per_vehicle_rail_freight,avgload_trns_freight_tonne_per_vehicle_road_heavy_freight,avgload_trns_freight_tonne_per_vehicle_water_borne,avgmass_lvst_animal_buffalo_kg,avgmass_lvst_animal_cattle_dairy_kg,avgmass_lvst_animal_cattle_nondairy_kg,avgmass_lvst_animal_chickens_kg,...,ef_ippu_tonne_cf4_per_tonne_production_electronics,ef_ippu_tonne_cf4_per_tonne_production_metals,ef_ippu_tonne_c3f8_per_tonne_production_chemicals,ef_ippu_tonne_c3f8_per_tonne_production_electronics,ef_ippu_tonne_c4f10_per_mmm_gdp_product_use_ods_other,ef_ippu_tonne_c4f10_per_tonne_production_chemicals,ef_ippu_tonne_c6f14_per_mmm_gdp_product_use_ods_other,ef_ippu_tonne_c6f14_per_tonne_production_chemicals,ef_ippu_tonne_cc4f8_per_tonne_production_chemicals,ef_ippu_tonne_cc4f8_per_tonne_production_electronics
0,costa_rica,0,70.0,2923.0,31.751466,6468.0,322.900664,520.741388,310.599686,1.12759,...,0.000002,4.204879e-07,0.0,1.515048e-07,0.0,0.0,0.0,0.0,0.0,5.669821e-08
1,costa_rica,1,70.0,2923.0,31.751466,6468.0,322.900664,520.741388,310.599686,1.12759,...,0.000002,2.134675e-07,0.0,1.313925e-07,0.0,0.0,0.0,0.0,0.0,5.185820e-08
2,costa_rica,2,70.0,2923.0,31.751466,6468.0,322.900664,520.741388,310.599686,1.12759,...,0.000002,1.821067e-07,0.0,1.060066e-07,0.0,0.0,0.0,0.0,0.0,5.269348e-08
3,costa_rica,3,70.0,2923.0,31.751466,6468.0,322.900664,520.741388,310.599686,1.12759,...,0.000002,2.094712e-07,0.0,1.093024e-07,0.0,0.0,0.0,0.0,0.0,5.500450e-08
4,costa_rica,4,70.0,2923.0,31.751466,6468.0,322.900664,520.741388,310.599686,1.12759,...,0.000002,3.446161e-07,0.0,1.046451e-07,0.0,0.0,0.0,0.0,0.0,5.266081e-08


In [98]:
# Checks if there are any differences in columns between the example df and our df
g_utils.compare_dfs(df_inputs_example, df_inputs_raw)


Columns in df1(example) but not in df2(yours): {'nemomod_entc_output_activity_ratio_fuel_production_fp_ammonia_production_ammonia', 'frac_trns_fuelmix_water_borne_ammonia', 'nemomod_entc_fixed_cost_fp_hydrogen_reformation_ccs_mm_usd_per_gw', 'nemomod_entc_output_activity_ratio_fuel_production_fp_hydrogen_reformation_ccs_hydrogen', 'nemomod_entc_input_activity_ratio_fuel_production_fp_hydrogen_reformation_ccs_oil', 'nemomod_entc_frac_min_share_production_fp_hydrogen_reformation_ccs', 'nemomod_entc_emissions_activity_ratio_fuel_production_fp_hydrogen_reformation_ccs_tonne_co2_per_tj', 'cost_enfu_fuel_ammonia_usd_per_tonne', 'frac_enfu_fuel_demand_imported_pj_fuel_ammonia', 'nemomod_entc_input_activity_ratio_fuel_production_fp_ammonia_production_electricity', 'nemomod_entc_capital_cost_fp_hydrogen_reformation_ccs_mm_usd_per_gw', 'nemomod_entc_input_activity_ratio_fuel_production_fp_hydrogen_reformation_ccs_natural_gas', 'exports_enfu_pj_fuel_ammonia', 'nemomod_entc_grid_power_constraint_m

In [99]:
# Fixes differences and makes sure that our df is in the correct format.
# Note: Edit this if you need more changes in your df

df_inputs_raw = df_inputs_raw.rename(columns={'period':'time_period'})
df_inputs_raw_complete = g_utils.add_missing_cols(df_inputs_example, df_inputs_raw.copy())
df_inputs_raw_complete = df_inputs_raw_complete.drop(columns='iso_code3', errors='ignore')
df_inputs_raw_complete.head()

,region,pij_lndu_forests_primary_to_croplands,pij_lndu_forests_mangroves_to_wetlands,yf_agrc_nuts_tonne_ha,ef_lndu_conv_forests_secondary_to_other_gg_co2_ha,ef_lndu_conv_forests_secondary_to_wetlands_gg_co2_ha,ef_lndu_conv_other_to_forests_primary_gg_co2_ha,pij_lndu_forests_mangroves_to_forests_secondary,ef_lndu_conv_wetlands_to_settlements_gg_co2_ha,pij_lndu_grasslands_to_forests_secondary,...,nemomod_entc_input_activity_ratio_fuel_production_fp_hydrogen_electrolysis_water,nemomod_entc_input_activity_ratio_fuel_production_fp_hydrogen_reformation_ccs_electricity,energydensity_gravimetric_enfu_gj_per_tonne_fuel_ammonia,energydensity_gravimetric_enfu_gj_per_tonne_fuel_water,frac_trns_fuelmix_water_borne_ammonia,nemomod_entc_output_activity_ratio_fuel_production_fp_ammonia_production_ammonia,nemomod_entc_output_activity_ratio_fuel_production_fp_hydrogen_reformation_ccs_hydrogen,nemomod_entc_frac_min_share_production_fp_hydrogen_reformation_ccs,nemomod_entc_input_activity_ratio_fuel_production_fp_hydrogen_reformation_ccs_natural_gas,nemomod_entc_input_activity_ratio_fuel_production_fp_hydrogen_reformation_ccs_oil
0,uganda,0.009591,0,0.6913,0.083964,0.087912,0,0,0,0,...,0.000004,0,18.6,0.00005,0.0,1,1,0.0,1.315,0.0
1,uganda,0.011771,0,0.6497,0.083964,0.087912,0,0,0,0,...,0.000004,0,18.6,0.00005,0.0,1,1,0.0,1.315,0.0
2,uganda,0.011129,0,0.6995,0.083964,0.087912,0,0,0,0,...,0.000004,0,18.6,0.00005,0.0,1,1,0.0,1.315,0.0
3,uganda,0.011167,0,0.6757,0.083964,0.087912,0,0,0,0,...,0.000004,0,18.6,0.00005,0.0,1,1,0.0,1.315,0.0
4,uganda,0.011205,0,0.9438,0.083964,0.087912,0,0,0,0,...,0.000004,0,18.6,0.00005,0.0,1,1,0.0,1.315,0.0


In [100]:
# Double checking that our df is in the correct shape (Empty sets should be printed to make sure everything is Ok!)
g_utils.compare_dfs(df_inputs_example, df_inputs_raw_complete)

Columns in df1(example) but not in df2(yours): set()
Columns in df2(yours) but not in df1(example): {'nemomod_entc_variable_cost_pp_biomass_mm_usd_per_gw', 'nemomod_entc_variable_cost_pp_gas_mm_usd_per_mwh', 'nemomod_entc_variable_cost_pp_oil_mm_usd_per_gw', 'cost_enfu_fuel_biomass_usd_per_m3', 'nemomod_entc_variable_cost_pp_wind_mm_usd_per_gw', 'nemomod_entc_variable_cost_pp_ocean_mm_usd_per_mwh', 'demscalar_ippu_recycled_glass', 'nemomod_entc_variable_cost_pp_oil_mm_usd_per_mwh', 'nemomod_entc_variable_cost_pp_geothermal_mm_usd_per_gw', 'nemomod_entc_variable_cost_pp_geothermal_mm_usd_per_mwh', 'va_mining_mmm_usd', 'nemomod_entc_variable_cost_pp_gas_mm_usd_per_gw', 'nemomod_entc_variable_cost_pp_biogas_mm_usd_per_gw', 'nemomod_entc_variable_cost_pp_biomass_mm_usd_per_mwh', 'va_industrial_mmm_usd', 'demscalar_ippu_recycled_plastic', 'nemomod_entc_variable_cost_pp_biogas_mm_usd_per_mwh', 'demscalar_ippu_recycled_wood', 'nemomod_entc_variable_cost_pp_wind_mm_usd_per_mwh', 'nemomod_entc_

In [101]:
# Checking if there are any columns with null values in it
columns_with_na = df_inputs_raw_complete.columns[df_inputs_raw_complete.isna().any()].tolist()

print(columns_with_na)

['exports_enfu_pj_fuel_kerosene', 'frac_enfu_fuel_demand_imported_pj_fuel_kerosene', 'frac_enfu_fuel_demand_imported_pj_fuel_oil', 'exports_enfu_pj_fuel_coal', 'frac_enfu_fuel_demand_imported_pj_fuel_electricity', 'exports_enfu_pj_fuel_natural_gas', 'exports_enfu_pj_fuel_gasoline', 'exports_enfu_pj_fuel_oil', 'frac_enfu_fuel_demand_imported_pj_fuel_hydrocarbon_gas_liquids', 'exports_enfu_pj_fuel_electricity', 'exports_enfu_pj_fuel_diesel', 'frac_enfu_fuel_demand_imported_pj_fuel_natural_gas', 'frac_enfu_fuel_demand_imported_pj_fuel_coal', 'frac_enfu_fuel_demand_imported_pj_fuel_crude', 'frac_enfu_fuel_demand_imported_pj_fuel_gasoline', 'exports_enfu_pj_fuel_crude', 'exports_enfu_pj_fuel_hydrocarbon_gas_liquids', 'frac_enfu_fuel_demand_imported_pj_fuel_diesel']


In [102]:
# Checking region to avoid copy errors
df_inputs_raw_complete['region'].head()

0    uganda
1    uganda
2    uganda
3    uganda
4    uganda
Name: region, dtype: object

In [103]:
# Set region to country name
df_inputs_raw_complete['region'] = country_name
df_inputs_raw_complete['region'].head()

0    uganda
1    uganda
2    uganda
3    uganda
4    uganda
Name: region, dtype: object

In [104]:
df_inputs_raw_complete.head()

,region,pij_lndu_forests_primary_to_croplands,pij_lndu_forests_mangroves_to_wetlands,yf_agrc_nuts_tonne_ha,ef_lndu_conv_forests_secondary_to_other_gg_co2_ha,ef_lndu_conv_forests_secondary_to_wetlands_gg_co2_ha,ef_lndu_conv_other_to_forests_primary_gg_co2_ha,pij_lndu_forests_mangroves_to_forests_secondary,ef_lndu_conv_wetlands_to_settlements_gg_co2_ha,pij_lndu_grasslands_to_forests_secondary,...,nemomod_entc_input_activity_ratio_fuel_production_fp_hydrogen_electrolysis_water,nemomod_entc_input_activity_ratio_fuel_production_fp_hydrogen_reformation_ccs_electricity,energydensity_gravimetric_enfu_gj_per_tonne_fuel_ammonia,energydensity_gravimetric_enfu_gj_per_tonne_fuel_water,frac_trns_fuelmix_water_borne_ammonia,nemomod_entc_output_activity_ratio_fuel_production_fp_ammonia_production_ammonia,nemomod_entc_output_activity_ratio_fuel_production_fp_hydrogen_reformation_ccs_hydrogen,nemomod_entc_frac_min_share_production_fp_hydrogen_reformation_ccs,nemomod_entc_input_activity_ratio_fuel_production_fp_hydrogen_reformation_ccs_natural_gas,nemomod_entc_input_activity_ratio_fuel_production_fp_hydrogen_reformation_ccs_oil
0,uganda,0.009591,0,0.6913,0.083964,0.087912,0,0,0,0,...,0.000004,0,18.6,0.00005,0.0,1,1,0.0,1.315,0.0
1,uganda,0.011771,0,0.6497,0.083964,0.087912,0,0,0,0,...,0.000004,0,18.6,0.00005,0.0,1,1,0.0,1.315,0.0
2,uganda,0.011129,0,0.6995,0.083964,0.087912,0,0,0,0,...,0.000004,0,18.6,0.00005,0.0,1,1,0.0,1.315,0.0
3,uganda,0.011167,0,0.6757,0.083964,0.087912,0,0,0,0,...,0.000004,0,18.6,0.00005,0.0,1,1,0.0,1.315,0.0
4,uganda,0.011205,0,0.9438,0.083964,0.087912,0,0,0,0,...,0.000004,0,18.6,0.00005,0.0,1,1,0.0,1.315,0.0


#  Let's try building transformations using this


In [105]:
transformers = trf.transformers.Transformers(
    {},
    df_input = df_inputs_raw_complete,
)

##  Instantiate some transformations. Make sure to run this cell to create the transformations folder for the first time or if you wish to overwrite

In [106]:
# set an ouput path and instantiate
if not dir_transformations.exists():
    trf.instantiate_default_strategy_directory(
        transformers,
        dir_transformations,
    )
else:
    print(f"Directory {dir_transformations} already exists. Skipping instantiation.")


Directory /home/fabian_fuentes/repos/sisepuede_region_nbs_tony_ver/uganda/transformations already exists. Skipping instantiation.


##  --HERE, CUSTOMIZE YOUR TRANSFORMATIONS AND STRATEGIES--

### Customizing transformations and strategies files using TransformationUtils.py classes

In [107]:
# Generate new transformation files based on the excel mapping file. 
# Make sure to have the most updated format for the excel file, check the one used in this notebook for reference.

cw_file_path = os.path.join(dir_data, ssp_transformation_cw)
excel_yaml_handler = ExcelYAMLHandler(excel_file=cw_file_path, yaml_directory='transformations')

In [108]:
# This creates transformation yaml files for each strategy in the excel file

# Use if its the first time you are running this
#excel_yaml_handler.process_yaml_files()

# Use if you have already created the yaml files and just want to update them so the transformations with multiple params are not overwritten
# NOTE: This is a temporary solution, we need to find a better way to handle this
excel_yaml_handler.process_yaml_files(overwrite_mult_param_transformations=False)

YAML file transformation_agrc_inc_conservation_agriculture.yaml for strategy strategy_priority_mitigation wasn't updated. Please check it manually.
YAML file transformation_agrc_inc_conservation_agriculture.yaml for strategy strategy_aditional_actions wasn't updated. Please check it manually.
YAML file transformation_agrc_inc_conservation_agriculture.yaml for strategy strategy_net_zero wasn't updated. Please check it manually.
YAML file transformation_inen_shift_fuel_heat.yaml for strategy strategy_net_zero wasn't updated. Please check it manually.
YAML file transformation_lsmm_inc_management_cattle_pigs.yaml for strategy strategy_net_zero wasn't updated. Please check it manually.
YAML file transformation_lsmm_inc_management_other.yaml for strategy strategy_net_zero wasn't updated. Please check it manually.
YAML file transformation_lsmm_inc_management_poultry.yaml for strategy strategy_net_zero wasn't updated. Please check it manually.
YAML file transformation_lvst_dec_enteric_fermenta

In [109]:
# Load the transformations per strategy dictionary so we can pass it to the strategy handler
# You can also check here if the transformations in each strategy are correct
transformation_per_strategy_dict = excel_yaml_handler.get_transformations_per_strategy_dict()
transformation_per_strategy_dict

{'strategy_priority_mitigation': ['TX:AGRC:DEC_CH4_RICE_STRATEGY_PRIORITY_MITIGATION',
  'TX:AGRC:INC_CONSERVATION_AGRICULTURE_STRATEGY_PRIORITY_MITIGATION',
  'TX:AGRC:INC_PRODUCTIVITY_STRATEGY_PRIORITY_MITIGATION',
  'TX:ENTC:DEC_LOSSES_STRATEGY_PRIORITY_MITIGATION',
  'TX:ENTC:TARGET_CLEAN_HYDROGEN_STRATEGY_PRIORITY_MITIGATION',
  'TX:ENTC:TARGET_RENEWABLE_ELEC_STRATEGY_PRIORITY_MITIGATION',
  'TX:INEN:INC_EFFICIENCY_ENERGY_STRATEGY_PRIORITY_MITIGATION',
  'TX:INEN:INC_EFFICIENCY_PRODUCTION_STRATEGY_PRIORITY_MITIGATION',
  'TX:IPPU:DEC_CLINKER_STRATEGY_PRIORITY_MITIGATION',
  'TX:IPPU:DEC_DEMAND_STRATEGY_PRIORITY_MITIGATION',
  'TX:IPPU:DEC_HFCS_STRATEGY_PRIORITY_MITIGATION',
  'TX:LNDU:DEC_DEFORESTATION_STRATEGY_PRIORITY_MITIGATION',
  'TX:LNDU:DEC_SOC_LOSS_PASTURES_STRATEGY_PRIORITY_MITIGATION',
  'TX:LNDU:INC_REFORESTATION_STRATEGY_PRIORITY_MITIGATION',
  'TX:LNDU:INC_SILVOPASTURE_STRATEGY_PRIORITY_MITIGATION',
  'TX:LVST:INC_PRODUCTIVITY_STRATEGY_PRIORITY_MITIGATION',
  'TX:TRDE

### Creating new strategies
- You can create new strategies from scratch.
- You can also update existing ones.

In [110]:
# Creating new strategies by updating the strategy_definitions file.
# You can edit this to add yours, as many as you want.
csv_handler = StrategyCSVHandler(strategies_definitions_file_path, dir_transformations, strategy_mapping_file_path, transformation_per_strategy_dict)

# Uncomment for completly new strategies version
#csv_handler.add_strategy(strategy_group='PFLO', description='NDC Priority Mitigation', yaml_file_suffix='priority_mitigation')
#csv_handler.add_strategy(strategy_group='PFLO', description='NDC Aditional Actions', yaml_file_suffix='aditional_actions')


In [111]:
# Uncomment to update the strategies that are currently in the strategy_definitions file

csv_handler.add_strategy(strategy_group='PFLO', description='NDC Priority Mitigation', yaml_file_suffix='priority_mitigation', custom_id=6006, update_flag=True)
csv_handler.add_strategy(strategy_group='PFLO', description='NDC Aditional Actions', yaml_file_suffix='aditional_actions', custom_id=6007, update_flag=True)


Data saved to /home/fabian_fuentes/repos/sisepuede_region_nbs_tony_ver/uganda/transformations/strategy_definitions.csv
Updated row with strategy_id 6006
Data saved to /home/fabian_fuentes/repos/sisepuede_region_nbs_tony_ver/uganda/transformations/strategy_definitions.csv
Updated row with strategy_id 6007


In [112]:
# Set up the strategy codes you wish to run in ssp
strategies_to_run = [0, 6006, 6007, 6005]

### We finished adding new transformation files and strategies so lets load them back

In [113]:
# then, you can load this back in after modifying (play around with it)
transformations = trf.Transformations(
    dir_transformations,
    transformers = transformers,
)
tab = transformations.attribute_transformation.table

In [114]:
#  build the strategies -- will export to path
t0 = time.time()
strategies = trf.Strategies(
    transformations,
    export_path = "transformations",
    prebuild = True,
)

t_elapse = sf.get_time_elapsed(t0)
print(f"Strategies defined at {strategies.transformations.dir_init} initialized in {t_elapse} seconds")

Strategies defined at /home/fabian_fuentes/repos/sisepuede_region_nbs_tony_ver/uganda/transformations initialized in 3.07 seconds


In [115]:
strategies.attribute_table

,strategy_id,strategy_code,strategy,description,transformation_specification,baseline_strategy_id
0,0,BASE,Strategy TX:BASE,NaN,TX:BASE,1
1,1000,AGRC:DEC_CH4_RICE,Singleton - Default Value - AGRC: Improve rice...,NaN,TX:AGRC:DEC_CH4_RICE,0
2,1001,AGRC:DEC_EXPORTS,Singleton - Default Value - AGRC: Decrease Exp...,NaN,TX:AGRC:DEC_EXPORTS,0
3,1002,AGRC:DEC_LOSSES_SUPPLY_CHAIN,Singleton - Default Value - AGRC: Reduce suppl...,NaN,TX:AGRC:DEC_LOSSES_SUPPLY_CHAIN,0
4,1003,AGRC:INC_CONSERVATION_AGRICULTURE,Singleton - Default Value - AGRC: Expand conse...,NaN,TX:AGRC:INC_CONSERVATION_AGRICULTURE,0
...,...,...,...,...,...,...
69,6003,PFLO:NDC_2_SCENARIO,NDC_2_scenario,NDC Unconditional,NaN,0
70,6004,PFLO:NDC_1_SCENARIO,NDC_1_scenario,NDC 2 deg. Scenario,NaN,0
71,6005,PFLO:NET_ZERO,net_zero,NDC 1.5 deg. Scenario,TX:LSMM:INC_MANAGEMENT_CATTLE_PIGS_STRATEGY_NE...,0
72,6006,PFLO:PRIORITY_MITIGATION,priority_mitigation,NDC Priority Mitigation,TX:TRDE:DEC_DEMAND_STRATEGY_PRIORITY_MITIGATIO...,0


##  Build our templates
- let's use the default variable groupings for LHS

In [116]:
# Building excel templates, make sure to include the strategies ids in the strategies attribute as well as the baseline (0)
df_vargroups = examples("variable_trajectory_group_specification")

strategies.build_strategies_to_templates(
    df_trajgroup = df_vargroups,
    include_simplex_group_as_trajgroup = True,
    strategies = strategies_to_run,
)

0

# Finally, load SISEPUEDE so that we can run it

In [117]:
country_name

'uganda'

In [118]:
# Make sure to change the regions attribute to your country name

import sisepuede as si
# timestamp_str = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
ssp = si.SISEPUEDE(
    "calibrated",
    db_type = "csv",
    # id_str = f"sisepuede_run_2024-11-04T09:23:26.721580",
    initialize_as_dummy = False, # no connection to Julia is initialized if set to True
    regions = [country_name],
    strategies = strategies,
    try_exogenous_xl_types_in_variable_specification = True,
)

2025-01-23 19:57:02,042 - INFO - Successfully initialized SISEPUEDEFileStructure.
2025-01-23 19:57:02,042 - INFO - Successfully initialized SISEPUEDEFileStructure.
2025-01-23 19:57:02,042 - INFO - Successfully initialized SISEPUEDEFileStructure.
2025-01-23 19:57:02,047 - WARNING - Missing key dict_dimensional_keys: key time_series not found. Tables that rely on the time_series will not have index checking.
2025-01-23 19:57:02,047 - WARNING - Missing key dict_dimensional_keys: key time_series not found. Tables that rely on the time_series will not have index checking.
2025-01-23 19:57:02,047 - WARNING - Missing key dict_dimensional_keys: key time_series not found. Tables that rely on the time_series will not have index checking.
2025-01-23 19:57:02,048 - INFO - 	Setting export engine to 'csv'.
2025-01-23 19:57:02,048 - INFO - 	Setting export engine to 'csv'.
2025-01-23 19:57:02,048 - INFO - 	Setting export engine to 'csv'.
2025-01-23 19:57:02,050 - WARNING - No index fields defined. Ind

In [119]:
# This runs the model, make sure you edit key_stretegy with the strategy ids you want to execute include baseline (0)
dict_scens = {
    ssp.key_design: [0],
    ssp.key_future: [0],
    ssp.key_strategy: strategies_to_run,
}

ssp.project_scenarios(
    dict_scens,
    save_inputs = True,
)

2025-01-23 19:57:23,248 - INFO - 
***	STARTING REGION uganda	***

2025-01-23 19:57:23,248 - INFO - 
***	STARTING REGION uganda	***

2025-01-23 19:57:23,248 - INFO - 
***	STARTING REGION uganda	***

2025-01-23 19:57:25,816 - INFO - Trying run primary_id = 0 in region uganda
2025-01-23 19:57:25,816 - INFO - Trying run primary_id = 0 in region uganda
2025-01-23 19:57:25,816 - INFO - Trying run primary_id = 0 in region uganda
2025-01-23 19:57:25,818 - INFO - Running AFOLU model
2025-01-23 19:57:25,818 - INFO - Running AFOLU model
2025-01-23 19:57:25,818 - INFO - Running AFOLU model
2025-01-23 19:57:25,983 - INFO - AFOLU model run successfully completed
2025-01-23 19:57:25,983 - INFO - AFOLU model run successfully completed
2025-01-23 19:57:25,983 - INFO - AFOLU model run successfully completed
2025-01-23 19:57:25,985 - INFO - Running CircularEconomy model
2025-01-23 19:57:25,985 - INFO - Running CircularEconomy model
2025-01-23 19:57:25,985 - INFO - Running CircularEconomy model
2025-01-23

{'uganda': [0, 71071, 72072, 73073]}

In [29]:
# df_run = ssp.generate_scenario_database_from_primary_key(0)
# df_run = df_run.get("iran")
# df_run.to_csv("/Users/usuario/Desktop/iran_baselines_temp.csv", encoding = "UTF-8", index = None, )

In [30]:
# Read input and output files
df_out = ssp.read_output(None)
df_in = ssp.read_input(None)

## Some testing for Croatia (you can delete this section for a new country)

In [ ]:
df_out.head()

In [ ]:
# Checking we've got an output df
df_out[[col for col in df_out.columns if 'subsector' in col]].head()

In [ ]:
df_out[[col for col in df_out.columns if 'subsector' in col]].tail()

In [ ]:
cement_df = df_out[['primary_id', 'time_period'] + [col for col in df_out.columns if 'prod_ippu_cement_tonne' in col]]
cement_df['year'] = cement_df['time_period'] + 2015
cement_df[cement_df.primary_id == 0]

In [ ]:
mining_df = df_out[['primary_id', 'time_period'] + [col for col in df_out.columns if 'prod_ippu_mining_tonne' in col]]
mining_df['year'] = mining_df['time_period'] + 2015
mining_df[mining_df.primary_id == 0]

In [ ]:
df_inputs_raw_complete['lndu_reallocation_factor'].mean()

In [ ]:
df_in['lndu_reallocation_factor'].mean()

In [ ]:
df_inputs_raw_complete['elasticity_ippu_cement_production_to_gdp'].head()

In [ ]:
df_in[['primary_id', 'time_period', 'elasticity_ippu_cement_production_to_gdp']].head()

In [ ]:
[col for col in df_out.columns if 'lvst' in col]

In [ ]:
df_out[['primary_id', 'time_period'] + [col for col in df_out.columns if 'demand_lvst' in col]].head(36)

In [ ]:
df_out[['primary_id', 'time_period'] + [col for col in df_out.columns if 'pop_lvst' in col]].head(37)

In [ ]:
df_in[ 'elasticity_protein_in_diet_to_gdppc']

In [ ]:
df_in['frac_gnrl_eating_red_meat']

In [ ]:
df_in[df_in.time_period == 5][['time_period'] + [col for col in df_in.columns if 'pop_lvst' in col]].head(6)

In [ ]:
df_in[df_in.primary_id == 0][['frac_waso_non_recycled_incinerated', 'frac_waso_non_recycled_landfilled', 'frac_waso_non_recycled_open_dump']].head()

In [ ]:
df_qty_open_dump = df_out[[col for col in df_out.columns if 'qty_waso_open_dump' in col]]
df_qty_open_dump

In [ ]:
df_qty_open_dump.head(15).sum(axis=1)

In [ ]:
df_out[df_out.isna().any(axis=1)]

In [ ]:
df_in[[col for col in df_in.columns if 'frac_waso_recycled' in col]]

# option to pass `df_out` back to data_modifications
- do this if the outputs are needed to rescale input demand values to match IEA data

In [51]:
# (
#     df_out
#     .to_csv(
#         dir_data.joinpath("sisepuede_outputs_iran_preiea.csv"),
#         index = None,
#         encoding = "UTF-8",
#     )
# )

In [ ]:

fig, ax = plt.subplots(figsize = (18, 8))
ax.set_xlabel("Time Period")
ax.set_ylabel("MT Emissions CO2e ")

df_plot = df_out[
    df_out[ssp.key_primary].isin([71071])
]


fields = matt.get_all_subsector_emission_total_fields()#[x for x in df_out.columns if (x.startswith("emission_co2e_subsector_total"))]
dict_format = dict(
    (k, {"color": v}) for (k, v) in
    matt.get_subsector_color_map().items()
)

fig, ax = spu.plot_stack(
    df_plot,
    fields,
    dict_formatting = dict_format,
    field_x = "time_period",
    figtuple = (fig, ax),
)


# Export Wide File (Last Mandatory Step)

In [53]:
df_out = ssp.read_output(None)
df_in = ssp.read_input(None)


all_primaries = sorted(list(df_out[ssp.key_primary].unique()))

# build if unable to simply read the data frame
if df_in is None:
    df_in = []
     
    for region in ssp.regions:
        for primary in all_primaries: 
            df_in_filt = ssp.generate_scenario_database_from_primary_key(primary)
            df_in.append(df_in_filt.get(region))
    
    df_in = pd.concat(df_in, axis = 0).reset_index(drop = True)




df_export = pd.merge(
    df_out,
    df_in,
    how = "left",
)



# check output directory 
dir_pkg = os.path.join(
    ssp.file_struct.dir_out, 
    f"sisepuede_summary_results_run_{ssp.id_fs_safe}"
)
os.makedirs(dir_pkg) if not os.path.exists(dir_pkg) else None


for tab in ["ATTRIBUTE_STRATEGY"]:
    table_df = ssp.database.db.read_table(tab)
    if table_df is not None:
        table_df.to_csv(
            os.path.join(dir_pkg, f"{tab}.csv"),
            index=None,
            encoding="UTF-8"
        )
    else:
        print(f"Warning: Table {tab} returned None.")


df_primary = (
    ssp
    .odpt_primary
    .get_indexing_dataframe(
        sorted(list(df_out[ssp.key_primary].unique()))
    )
)
    
df_primary.to_csv(
    os.path.join(dir_pkg, f"ATTRIBUTE_PRIMARY.csv"),
    index = None,
    encoding = "UTF-8"
)

df_export.to_csv(
    os.path.join(dir_pkg, f"sisepuede_results_{ssp.id_fs_safe}_WIDE_INPUTS_OUTPUTS.csv"),
    index = None,
    encoding = "UTF-8"
)

In [ ]:
# Getting the directory where the outputs are stored
ssp.file_struct.dir_out

## Some Optional Plotting

In [56]:
# primary_id = 0

# df_prim = df_out[
#     df_out[
#         ssp.key_primary
#     ]
#     .isin([primary_id])
# ]

# df_in_cur = df_in[
#     df_in[
#         ssp.key_primary
#     ]
#     .isin([primary_id])
# ]


# fields_plot = [
#     x for x in df_prim.columns
#     if "investment" in x
#     and df_prim[x].max() > 0
# ]
# (
#     df_prim[fields_plot]
#     .plot(
#         legend = True,
#     )
# )




In [57]:


# fig, ax = plt.subplots(figsize = (15, 10))
# df_resid = matt.extract_model_variable(
#     df_in_cur,
#     "NemoMod ResidualCapacity"
# )
# (
#     df_resid
#     .plot
#     .area(
#         ax = ax
#     )
# )

In [58]:
# df_resid

In [59]:
# [x for x in df_out.columns if "capacity" in x]

In [60]:
# ssp.model_attributes.configuration.dict_config


In [61]:
# modvar = matt.get_variable("NemoMod Generation Capacity")
# modvar.get_from_dataframe(df_prim).head()

In [62]:
# #[x for x in matt.all_variables if "Consumption" in x]
# [x for x in matt.all_variables if "Production" in x]
# modvar = matt.get_variable("NemoMod Production by Technology")


# unit = matt.get_unit("energy")
# (modvar.get_from_dataframe(df_prim).iloc[7].sum()*unit.convert("PJ", "mwh")/1000)/367669

In [63]:
# df_inputs

In [64]:
# modvar = matt.get_variable("Total Energy Consumption from Industrial Energy")#Industrial Energy")
# matt.extract_model_variable(
#     df_prim,
#     modvar
# )